In [1]:
from google.colab import drive
drive.mount('/content/gdrive',force_remount=True)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


# Imported importanat libraries

In [8]:
# baseline model with dropout and data augmentation on the cifar10 dataset
import sys
import tensorflow
from matplotlib import pyplot
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential,Model
from tensorflow.keras.layers import Conv2D,MaxPooling2D,Dense,Flatten,Dropout,BatchNormalization,AveragePooling2D
from tensorflow.keras.optimizers import SGD,Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.preprocessing.image import load_img
from imutils import paths
import matplotlib.pyplot as plt
import numpy as np
import random
import os

# Preprocessing of image and labels

In [3]:
data_train = '/content/gdrive/My Drive/emotion_recog/emo'

In [4]:
imagepaths = list(paths.list_images(data_train))
random.shuffle(imagepaths)
random.seed(42)
data_images = []
labels = []
for imgpath in imagepaths:
	label = imgpath.split(os.path.sep)[-2]
	img = load_img(imgpath, target_size=(224, 224))
	data_images.append(img_to_array(img)/255.)
	labels.append(label) 

/usr/local/lib/python3.6/dist-packages/PIL/Image.py:932: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  "Palette images with Transparency expressed in bytes should be "


In [5]:
from sklearn.preprocessing import LabelEncoder
lb = LabelEncoder()
labels_z = lb.fit_transform(np.array(labels))
labels_value = to_categorical(labels_z,num_classes=6)
data = np.array(data_images,dtype='float32')

# train and test splitting with sklearn


In [6]:
trainX, testX,trainY, testY = train_test_split(data, labels_value,test_size=0.2, random_state=42,shuffle = True)

# Transfer learning - Mobilenet-v2

In [9]:
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import Input
baseModel = MobileNetV2(weights="imagenet", include_top=False,input_tensor=Input(shape=(224, 224, 3)))

# construct the head of the model that will be placed on top of the
# the base model
headModel = baseModel.output
headModel = AveragePooling2D(pool_size=(7, 7))(headModel)
headModel = Flatten(name="flatten")(headModel)
headModel = Dense(128, activation="relu")(headModel)
headModel = Dropout(0.5)(headModel)
headModel = Dense(6, activation="softmax")(headModel)
model2 = Model(inputs=baseModel.input, outputs=headModel)

# loop over all layers in the base model and freeze them so they will
# *not* be updated during the first training process
for layer in baseModel.layers:
	layer.trainable = False

# compile our model
INIT_LR = 1e-4
print("[INFO] compiling model...")
opt = Adam(lr=INIT_LR, decay=INIT_LR /100)
model2.compile(loss="categorical_crossentropy", optimizer=opt,
	metrics=["accuracy"])


[INFO] compiling model...


In [16]:
model2.summary()

Model: "functional_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
Conv1_pad (ZeroPadding2D)       (None, 225, 225, 3)  0           input_2[0][0]                    
__________________________________________________________________________________________________
Conv1 (Conv2D)                  (None, 112, 112, 32) 864         Conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_Conv1 (BatchNormalization)   (None, 112, 112, 32) 128         Conv1[0][0]                      
_______________________________________________________________________________________

# Training and plotting accuracy and loss

In [10]:
# plot diagnostic learning curves
import matplotlib.pyplot as plt
def summarize_diagnostics(hist):
	plt.plot(hist.history["accuracy"])
	plt.plot(hist.history['val_accuracy'])
	plt.plot(hist.history['loss'])
	plt.plot(hist.history['val_loss'])
	plt.title("model accuracy")
	plt.ylabel("Accuracy")
	plt.xlabel("Epoch")
	plt.legend(["Accuracy","Validation Accuracy","loss","Validation Loss"])
	plt.show()

In [11]:
# run the test harness for evaluating a model
def run_test_harness():
	# create data generator
	datagen = ImageDataGenerator(width_shift_range=0.1, height_shift_range=0.1, horizontal_flip=True,zoom_range=0.1)
	# prepare iterator
	it_train = datagen.flow(trainX, trainY, batch_size=32)
	# fit model
	steps = int(trainX.shape[0] / 32)
	hist = model2.fit_generator(it_train, steps_per_epoch=steps, epochs=100, validation_data=(testX, testY), verbose=1)
	# evaluate model
	_, acc = model2.evaluate(testX, testY, verbose=1)
	print('> %.3f' % (acc * 100.0))
	# learning curves
	summarize_diagnostics(hist)

# entry point, run the test harness
run_test_harness()

Instructions for updating:
Please use Model.fit, which supports generators.
Epoch 1/100
53/53 [==============================] - 18s 340ms/step - loss: 1.7525 - accuracy: 0.3414 - val_loss: 1.2672 - val_accuracy: 0.5174
Epoch 2/100
53/53 [==============================] - 17s 325ms/step - loss: 1.2913 - accuracy: 0.4970 - val_loss: 1.0964 - val_accuracy: 0.5847
Epoch 3/100
53/53 [==============================] - 17s 324ms/step - loss: 1.1227 - accuracy: 0.5609 - val_loss: 0.9823 - val_accuracy: 0.6311
Epoch 4/100
53/53 [==============================] - 17s 322ms/step - loss: 1.0413 - accuracy: 0.5923 - val_loss: 0.9403 - val_accuracy: 0.6357
Epoch 5/100
53/53 [==============================] - 17s 324ms/step - loss: 0.9568 - accuracy: 0.6254 - val_loss: 0.8875 - val_accuracy: 0.6589
Epoch 6/100
53/53 [==============================] - 17s 323ms/step - loss: 0.9290 - accuracy: 0.6402 - val_loss: 0.8830 - val_accuracy: 0.6566
Epoch 7/100
53/53 [==============================] - 17s 327

NameError: ignored

# Confusion matrix and classification report

In [12]:
from sklearn.metrics import confusion_matrix,classification_report
y_pred = model2.predict(testX)
y_p = np.argmax(y_pred,axis=1)
y_true = np.argmax(testY,axis=1)
print(confusion_matrix(y_true,y_p))

[[30  0  0  0  0  0]
 [ 0 51 19 22  6  0]
 [ 0  9 64  6  1  0]
 [ 0  3  7 56  4  0]
 [ 0  5 13  9 31  0]
 [ 0  0  0  0  0 95]]


In [13]:
print('Classification report')
print(classification_report(y_true,y_p))

Classification report
              precision    recall  f1-score   support

           0       1.00      1.00      1.00        30
           1       0.75      0.52      0.61        98
           2       0.62      0.80      0.70        80
           3       0.60      0.80      0.69        70
           4       0.74      0.53      0.62        58
           5       1.00      1.00      1.00        95

    accuracy                           0.76       431
   macro avg       0.79      0.78      0.77       431
weighted avg       0.77      0.76      0.75       431



# CNN + SVM

In [19]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
# extracting features using Transfer Learning
model_new = Model(inputs = model2.input,outputs = model2.get_layer('dense_2').output) 
train_new = sc.fit_transform(model_new.predict(trainX))
test_new = sc.fit_transform(model_new.predict(testX))

In [20]:

from sklearn.svm import SVC
svm = SVC(kernel='rbf')
svm.fit(train_new,np.argmax(trainY,axis=1))
sc1 = svm.score(train_new,np.argmax(trainY,axis=1))
sc2 = svm.score(test_new,np.argmax(testY,axis=1))
print('training accuracy of svm is : ',sc1)
print('testing accuracy of svm is : ',sc2)

training accuracy of svm is :  0.9883855981416957
testing accuracy of svm is :  0.8004640371229699


# CNN + XGBOOST


In [21]:
from xgboost import XGBClassifier
xg = XGBClassifier()
xg.fit(train_new,np.argmax(trainY,axis=1))
sc3 = xg.score(train_new,np.argmax(trainY,axis=1))
sc4 = xg.score(test_new,np.argmax(testY,axis=1))
print('training accuracy of xgboost is : ',sc3)
print('testing accuracy of xgboost is : ',sc4)

training accuracy of xgboost is :  0.9988385598141696
testing accuracy of xgboost is :  0.7842227378190255
